# Speech Emotion Recognition - Feature Extraction

Databases used

* The Ryerson Audio-Visual Database of Emotional Speech and Song (RAVDESS)
* Toronto emotional speech set (TESS)

### Import Libraries

Import necessary libraries

In [1]:
import glob
import os
import librosa
import time
import numpy as np
import pandas as pd

### Load all files

We will create our numpy array extracting Mel-frequency cepstral coefficients (MFCCs) while the classes to predict will be extracted from the name of the file.

#### Defining emotions to classify 

Selecting the emotions to be classified. Note that the emotions 'neutral', 'calm' and 'surprised' are only found in RAVDESS dataset and 'pleasantly surprised' or 'ps' is only available in TESS dataset. To combine all the emotions from both datasets into 8 emotion classes, we have changed 'pleasantly surprised' to 'surprised' and 'fearful' to 'fear'

In [2]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fear',
  '07':'disgust',
  '08':'surprised'
}

#defined tess emotions to test on TESS dataset only
tess_emotions=['angry','disgust','fear','ps','happy','sad']

##defined RAVDESS emotions to test on RAVDESS dataset only
ravdess_emotions=['neutral','calm','angry', 'happy','disgust','sad','fear','surprised']

observed_emotions = ['sad','angry','happy','disgust','surprised','neutral','calm','fear']

#### Feature extraction

Using librosa package we can extract the MFCC features. This function loads the file give the file path and after resampling and computing MFCC features, returns the features. We have selected the no. of MFCCs as 40.

https://librosa.org/librosa/generated/librosa.feature.mfcc.html

In [3]:
def extract_feature(file_name, mfcc):
    X, sample_rate = librosa.load(os.path.join(file_name), res_type='kaiser_fast')
    
    result = np.array([])  # Initialize result as an empty array
    
    if mfcc:
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result = np.hstack((result, mfccs))  # Append MFCC features to result
    
    return result


#### Choosing a dataset

Choose the dataset(s) you want to load using the following function

In [4]:
def dataset_options():
    # choose datasets
    ravdess = True
    tess = True
    ravdess_speech = False
    ravdess_song = False
    data = {'ravdess':ravdess, 'ravdess_speech':ravdess_speech, 'ravdess_song':ravdess_song, 'tess':tess}
    print(data)
    return data

#### Load data

Load data from the datasets required which is obtained by calling the function dataset__options(). Extract features from each file with the selected emotions in chosen datasets using the extract_feature() function defined.

In [5]:
def dataset_options():
    # choose datasets
    ravdess = True
    tess = True
    ravdess_speech = False
    ravdess_song = False
    data = {'ravdess': ravdess, 'ravdess_speech': ravdess_speech, 'ravdess_song': ravdess_song, 'tess': tess}
    print(data)
    return data


def load_data(test_size=0.2):
    import os
    import glob

    x, y = [], []

    # Feature to extract
    mfcc = True

    data = dataset_options()

    ravdess_base_path = r"C:\\Users\\vaibh\\Desktop\\Final Major Project\\Machine Learning Algorithm\\Datasets\\RAVDESS\\"
    tess_base_path = r"C:\\Users\\vaibh\\Desktop\\Final Major Project\\Machine Learning Algorithm\\Datasets\\"

    paths = []

    if data['ravdess']:
        paths.append(os.path.join(ravdess_base_path, "*", "Actor_*", "*.wav"))
    elif data['ravdess_speech']:
        paths.append(os.path.join(ravdess_base_path, "Speech", "Actor_*", "*.wav"))
    elif data['ravdess_song']:
        paths.append(os.path.join(ravdess_base_path, "Song", "Actor_*", "*.wav"))

    for path in paths:
        for file in glob.glob(path):
            file_name = os.path.basename(file)
            emotion = emotions[file_name.split("-")[2]]
            if emotion not in observed_emotions:
                continue
            feature = extract_feature(file, mfcc)
            x.append(feature)
            y.append(emotion)

    if data['tess']:
        for file in glob.glob(os.path.join(tess_base_path, "*AF_*", "*.wav")):
            file_name = os.path.basename(file)
            emotion = file_name.split("_")[2][:-4]
            if emotion == 'ps':
                emotion = 'surprised'
            if emotion not in observed_emotions:
                continue
            feature = extract_feature(file, mfcc)
            x.append(feature)
            y.append(emotion)

    return {"X": x, "y": y}


In [6]:
# pip install resampy

In [7]:
start_time = time.time()

Trial_dict = load_data(test_size = 0.3)

print("--- Data loaded. Loading time: %s seconds ---" % (time.time() - start_time))

{'ravdess': True, 'ravdess_speech': False, 'ravdess_song': False, 'tess': True}
--- Data loaded. Loading time: 116.51460266113281 seconds ---


In [8]:
X = pd.DataFrame(Trial_dict["X"])
y = pd.DataFrame(Trial_dict["y"])

In [9]:
X.shape, y.shape

((2452, 40), (2452, 1))

In [10]:
#renaming the label column to emotion
y=y.rename(columns= {0: 'emotion'})

In [11]:
#concatinating the attributes and label into a single dataframe
data = pd.concat([X, y], axis =1)

In [12]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,emotion
0,-592.519714,55.269863,-19.240614,14.985035,-4.303793,-0.782968,-15.683035,-4.746462,-22.302805,-5.347614,...,-6.584931,-2.434462,-3.743651,-0.299833,1.267962,5.327148,3.982636,8.654726,11.515517,neutral
1,-571.771179,53.821236,-19.770306,14.672212,-3.221300,-2.219379,-15.660699,-4.115021,-21.762812,-0.923150,...,-6.433335,-2.461544,-4.143042,0.553830,1.998689,5.020371,4.224606,9.785756,11.845788,neutral
2,-579.216675,54.232811,-15.242695,17.250660,-4.751227,0.021345,-13.251750,-5.477705,-20.139534,-4.198607,...,-5.051620,-1.963996,-3.542640,-0.745385,1.221582,3.439565,5.602417,9.031712,9.748875,neutral
3,-581.090637,53.249722,-19.030127,17.035284,-5.648746,1.875090,-16.216503,-5.108720,-21.046432,-2.953222,...,-5.261817,-2.072452,-4.843050,0.880779,1.297034,4.160462,5.900162,9.407314,10.992938,neutral
4,-592.520569,61.175182,-16.242025,16.972326,-1.740968,-0.389398,-15.755614,-1.111939,-19.680891,-3.488038,...,2.623967,3.312066,-4.676118,-4.766972,-4.045164,1.991833,5.520200,10.280448,11.302133,calm


## Shuffling data

In [13]:
#reindexing to shuffle the data at random
data = data.reindex(np.random.permutation(data.index))

In [14]:
# Storing shuffled ravdess and tess data to avoid loading again
data.to_csv("RAVTESS_MFCC_Observed.csv")